<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [6]:
!pip install scipy --user

  Using cached https://files.pythonhosted.org/packages/be/cc/6f7842a4d9aa7f51155f849185631e1201df255742de84d724ac33bff723/scipy-1.3.0-cp37-cp37m-win32.whl


In [17]:
import numpy as np

class Neural_Network(object):
    def __init__(self):        
        #Define Hyperparameters
        self.inputLayerSize = 3
        self.outputLayerSize = 1
        self.hiddenLayerSize = 4
        
        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize, self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize, self.outputLayerSize)
        
    def forward(self, X):
        #Propogate inputs though network
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        yHat = self.sigmoid(self.z3) 
        return yHat
        
    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)
        return J
        
    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forward(X)
        
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(X.T, delta2)  
        
        return dJdW1, dJdW2
    
    #Helper Functions for interacting with other classes:
    def getParams(self):
        #Get W1 and W2 unrolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single paramater vector.
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, X, y):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [22]:
from scipy import optimize
class trainer(object):
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N
        
    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X, self.y))   
        
    def costFunctionWrapper(self, params, X, y):
        self.N.setParams(params)
        cost = self.N.costFunction(X, y)
        grad = self.N.computeGradients(X,y)
        
        return cost, grad
        
    def train(self, X, y):
        #Make an internal variable for the callback function:
        self.X = X
        self.y = y

        #Make empty list to store costs:
        self.J = []
        
        params0 = self.N.getParams()

        options = {'maxiter': 10000, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS', \
                                 args=(X, y), options=options, callback=self.callbackF)

        self.N.setParams(_res.x)
        self.optimizationResults = _res

In [31]:
X = np.array(([0, 0, 1],
              [0, 1, 1],
              [1, 0, 1],
              [0, 1, 0],
              [1, 0, 0],
              [1, 1, 1], 
              [0, 0, 0]), dtype=float)

y = np.array(([0],
              [1],
              [1],
              [1],
              [1],
              [0],
              [0]), dtype=float)

nn = Neural_Network()
T = trainer(nn)

T.train(X, y)

Optimization terminated successfully.
         Current function value: 0.500000
         Iterations: 46
         Function evaluations: 49
         Gradient evaluations: 49


In [28]:
[round(x[0]) for x in nn.forward(X)]

[0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]

In [32]:
nn.forward(X)

array([[4.56790385e-25],
       [1.00000000e+00],
       [9.99816876e-01],
       [1.00000000e+00],
       [9.99490998e-01],
       [1.00000000e+00],
       [4.86735585e-12]])

In [12]:
y

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.]])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [14]:
!pip install python-mnist --user

In [35]:
!pip install sklearn --user

In [53]:
import pandas as pd
from sklearn.datasets import load_digits
digits = load_digits()
X = digits.data
X = np.array([np.append(row, 1) for row in X])
y = digits.target

In [19]:
df = pd.DataFrame(y, columns=['digit'])
cols = list(range(10))

for col in cols:
    df[str(col)] = (df['digit'] == col).astype(int)
df = df.drop(columns=['digit'])

In [29]:
class Neural_Network_MNIST:
    def __init__(self, inputs=2, hiddenNodes=3, outputNodes=1):
        # Set up Architetecture
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes

        #Initial weights
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes) #2x3
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes) #3x1

    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))

    def sigmoidPrime(self, s):
        return s * (1 - s)

    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """

        #Weighted sume of inputs and hidden layer
        self.hidden_sum = np.dot(X, self.weights1) 

        #Acivations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)

        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)

        #Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)

        return self.activated_output

    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        self.o_error = y - o #error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid to error

        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)

        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [54]:
model = Neural_Network_MNIST(inputs=65, hiddenNodes=65, outputNodes=1)
for i in range(1000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 200 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
#         print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(model.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - model.feed_forward(X)))))
    model.train(X, y.reshape(-1,1))

+---------EPOCH 1---------+
Actual Output: 
 [0 1 2 ... 8 9 8]
Predicted Output: 
 [[0.99999986]
 [0.99999799]
 [0.99999978]
 ...
 [0.99999869]
 [0.99999969]
 [0.99999845]]
Loss: 
 20.391223341505345
+---------EPOCH 2---------+
Actual Output: 
 [0 1 2 ... 8 9 8]
Predicted Output: 
 [[0.9999999 ]
 [0.99999867]
 [0.99999985]
 ...
 [0.99999919]
 [0.99999981]
 [0.99999901]]
Loss: 
 20.391218282198793
+---------EPOCH 3---------+
Actual Output: 
 [0 1 2 ... 8 9 8]
Predicted Output: 
 [[0.99999992]
 [0.999999  ]
 [0.99999989]
 ...
 [0.99999941]
 [0.99999987]
 [0.99999926]]
Loss: 
 20.39121579839182
+---------EPOCH 4---------+
Actual Output: 
 [0 1 2 ... 8 9 8]
Predicted Output: 
 [[0.99999994]
 [0.99999919]
 [0.99999991]
 ...
 [0.99999954]
 [0.99999989]
 [0.99999941]]
Loss: 
 20.39121428301617
+---------EPOCH 5---------+
Actual Output: 
 [0 1 2 ... 8 9 8]
Predicted Output: 
 [[0.99999995]
 [0.99999933]
 [0.99999993]
 ...
 [0.99999963]
 [0.99999991]
 [0.99999951]]
Loss: 
 20.391213252066976
+-

In [43]:
preds = model.feed_forward(X)
from sklearn.metrics import accuracy_score

accuracy_score(preds.astype(int), df['1'].astype(int))

0.8987200890372844

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?